In [1]:
import cobra.io
model = cobra.io.read_sbml_model('Roseobacter-litoralis-strain-B14.SBML')

In [2]:
medium = model.medium

In [3]:
mediumlist = list(medium.keys())

In [4]:
mediumlist.remove('EX_glc__D_e')

In [5]:
for EX in mediumlist:
    medium[EX] = 2.5
    model.medium = medium

In [6]:
mediumwglc = list(medium.keys())

In [7]:
csources = list()

In [8]:
for reactionname in mediumwglc:
    reaction = model.reactions.get_by_id(reactionname)
    metabolite = reaction.reactants[0]
    comp = metabolite.formula
    checkcarbon = 'C' in comp
    if checkcarbon == True:
        csources.append(reaction.id)

In [9]:
csources.remove('EX_cl_e')
csources.remove('EX_ca2_e')
csources.remove('EX_cobalt2_e')
csources.remove('EX_cu2_e')

In [10]:
medium1 = model.medium
for Csource in csources:
    medium1[Csource]=0
model.medium = medium1
print(model.medium)

{'EX_h2o_e': 2.5, 'EX_h_e': 2.5, 'EX_cl_e': 2.5, 'EX_pi_e': 2.5, 'EX_nh4_e': 2.5, 'EX_fe3_e': 2.5, 'EX_aso3_e': 2.5, 'EX_k_e': 2.5, 'EX_ca2_e': 2.5, 'EX_mg2_e': 2.5, 'EX_mn2_e': 2.5, 'EX_cobalt2_e': 2.5, 'EX_zn2_e': 2.5, 'EX_cu2_e': 2.5, 'EX_o2_e': 2.5, 'EX_fe2_e': 2.5, 'EX_h2s_e': 2.5, 'EX_mobd_e': 2.5, 'EX_so4_e': 2.5}


In [11]:
newcsources = ['EX_ala__L_e','EX_arg__L_e','EX_asp__L_e', 'EX_glu__L_e', 'EX_gln__L_e', 
               'EX_gly_e', 'EX_his__L_e', 'EX_ile__L_e','EX_leu__L_e', 'EX_lys__L_e', 'EX_met__L_e', 
               'EX_orn_e', 'EX_pro__L_e', 'EX_ser__D_e', 'EX_thr__L_e', 'EX_val__L_e', 'EX_cellb_e', 
               'EX_arab__D_e', 'EX_arab__L_e', 'EX_fru_e', 'EX_fuc__L_e', 'EX_gal_e', 'EX_gam_e',
               'EX_glc__D_e', 'EX_lcts_e', 'EX_lac__D_e', 'EX_lac__L_e', 'EX_malt_e', 'EX_mnl_e', 
               'EX_man_e', 'EX_melib_e', 'EX_rmn_e', 'EX_rib__D_e','EX_sucr_e', 'EX_sbt__D_e', 
               'EX_tre_e', 'EX_xyl__D_e', 'EX_chol_e', 'EX_glyb_e', 'EX_ptrc_e', 'EX_taur_e', 
               'EX_etoh_e', 'EX_glyc_e', 'EX_urea_e']

Missing in newsources: asparagine, citrulline, cysteine, phenylalanine, tryptophan, tyrosine, creatine, creatinine, dimethylglycine, sarcosine, glycogen

In [12]:
for newcsource in newcsources:
    model.medium = medium1
    medium2 = model.medium
    medium2[newcsource]=10
    model.medium = medium2
    mumax = model.optimize().objective_value
    print(newcsource,mumax)

EX_ala__L_e 0.0
EX_arg__L_e 0.11832027929981208
EX_asp__L_e 0.0
EX_glu__L_e 0.0
EX_gln__L_e 0.0
EX_gly_e 0.0
EX_his__L_e 0.0
EX_ile__L_e 0.0
EX_leu__L_e 0.0
EX_lys__L_e 0.0
EX_met__L_e 0.0
EX_orn_e 0.0
EX_pro__L_e 0.0
EX_ser__D_e 0.0
EX_thr__L_e 0.0
EX_val__L_e 0.0
EX_cellb_e 0.0
EX_arab__D_e 0.0
EX_arab__L_e 0.0
EX_fru_e 0.0
EX_fuc__L_e 0.0
EX_gal_e 0.0
EX_gam_e 0.0
EX_glc__D_e 0.0
EX_lcts_e 0.0
EX_lac__D_e 0.0
EX_lac__L_e 0.0
EX_malt_e 0.0
EX_mnl_e 0.0
EX_man_e 0.0
EX_melib_e 0.0
EX_rmn_e 0.0
EX_rib__D_e 0.0
EX_sucr_e 0.0
EX_sbt__D_e 0.0
EX_tre_e 0.0
EX_xyl__D_e 0.0
EX_chol_e 0.0
EX_glyb_e 0.0
EX_ptrc_e 0.0
EX_taur_e 0.0
EX_etoh_e 0.0
EX_glyc_e 0.0
EX_urea_e 0.0


In [13]:
# Only carbon source it grows on is arginine

In [14]:
allex = model.exchanges

In [15]:
nsources = list()
for reaction in allex:
    metabolite = reaction.reactants[0]
    comp = metabolite.formula
    checkcarbon = 'N' in comp
    if checkcarbon == True:
        nsources.append(reaction.id)

In [16]:
for nsource in nsources:
    model.medium=medium1
    medium3 = model.medium
    medium3['EX_glc__D_e']=10
    medium3[nsource] = 10
    model.medium = medium3
    gr = model.optimize().objective_value
    if gr != 0:
        print(gr,nsource)

0.24756662199071044 EX_arg__L_e


In [17]:
# Only nitrogen source it grows on is arginine

In [18]:
#!carve --refseq GCF_014337925.1 --output Roseobacter-litoralis-strain-B14-M9.xml --gapfill M9 --init M9

In [19]:
M9model = cobra.io.read_sbml_model('Roseobacter-litoralis-strain-B14-M9.xml')

In [20]:
M9model.medium

{'EX_glc__D_e': 10.0,
 'EX_h2o_e': 10.0,
 'EX_h_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_pi_e': 10.0,
 'EX_nh4_e': 10.0,
 'EX_fe3_e': 10.0,
 'EX_k_e': 10.0,
 'EX_ca2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_mn2_e': 10.0,
 'EX_cobalt2_e': 10.0,
 'EX_zn2_e': 10.0,
 'EX_cu2_e': 10.0,
 'EX_o2_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_mobd_e': 10.0,
 'EX_so4_e': 10.0}

In [21]:
M9model.optimize().objective_value

0.7497699143341323

In [22]:
medium = M9model.medium

In [23]:
medium['EX_glc__D_e'] = 0

In [24]:
M9model.medium = medium

In [25]:
#Urea exchange not in M9 model

In [26]:
newcsources.remove('EX_urea_e')

Missing in newsources: asparagine, citrulline, cysteine, phenylalanine, tryptophan, tyrosine, creatine, creatinine, dimethylglycine, sarcosine, glycogen, urea

In [27]:
import pandas as pd

In [28]:
growthmodel = list()

In [29]:
for newcsource in newcsources:
    M9model.medium = medium
    medium1 = M9model.medium
    medium1[newcsource]=10
    M9model.medium = medium1
    mumax = M9model.optimize().objective_value
    if mumax != 0.0:
        growthmodel.append(1)
    else:
        growthmodel.append(0)

In [30]:
csourcegrowthmodel = pd.DataFrame({'Carbon source': newcsources, 'Growth': growthmodel})

In [31]:
csourcegrowthmodel

,Carbon source,Growth
0,EX_ala__L_e,1
1,EX_arg__L_e,1
2,EX_asp__L_e,1
3,EX_glu__L_e,1
4,EX_gln__L_e,1
5,EX_gly_e,1
6,EX_his__L_e,1
7,EX_ile__L_e,0
8,EX_leu__L_e,0
9,EX_lys__L_e,1


In [32]:
growthexp = [1,1,1,0,1,1,1,0,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1,0,0,1,1,0,1,1,1,1,1,0,0,0,1,1,1,1,0,1,1]

In [33]:
csourcegrowthexp = pd.DataFrame({'Carbon source': newcsources, 'Growth': growthexp})

In [34]:
csourcegrowthexp

,Carbon source,Growth
0,EX_ala__L_e,1
1,EX_arg__L_e,1
2,EX_asp__L_e,1
3,EX_glu__L_e,0
4,EX_gln__L_e,1
5,EX_gly_e,1
6,EX_his__L_e,1
7,EX_ile__L_e,0
8,EX_leu__L_e,0
9,EX_lys__L_e,0


In [42]:
TP = 0
FP = 0
TN = 0
FN = 0
for i in range(42):
    exp = csourcegrowthexp.iloc[i,1]
    mod = csourcegrowthmodel.iloc[i,1]
    if exp == 1:
        if mod == 1:
            TP = TP + 1
        elif mod == 0:
            FN = FN + 1
    if exp == 0:
        if mod == 0:
            TN = TN + 1
        elif mod == 1:
            FP = FP +1

In [44]:
print(TP)
print(TN)
print(FP)
print(FN)

23
6
10
3


In [41]:
range(42)

range(0, 42)

In [47]:
import math

In [50]:
Precision = TP/(TP+FP) #Positive predictive value

In [51]:
Precision #From 0 to 1, 1 being perfect prediction

0.696969696969697

In [58]:
NPV = TN/(TN+FN)

In [59]:
NPV

0.6666666666666666

In [60]:
Sensitivity = TP/(TP+FN)

In [61]:
Sensitivity

0.8846153846153846

In [62]:
Specificity = TN/(TN+FP)

In [63]:
Specificity

0.375

In [64]:
Accuracy = (TP+TN)/(TP+TN+FP+FN)

In [65]:
Accuracy

0.6904761904761905

In [66]:
F1score = 2*Precision*Sensitivity/(Precision+Sensitivity)

In [67]:
F1score #From 0 to 1, 1 being perfect prediction

0.7796610169491526

In [68]:
MCC = (TP*TN-FP*FN)/(math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)))

In [69]:
MCC  #from -1 to 1, where 0 would be random guessing

0.3072549338995135